In [1]:
!pip install accelerate

In [2]:
!pip install transformers pandas datasets

In [3]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset
from torch.utils.tensorboard import SummaryWriter

2024-04-21 21:13:07.219690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 21:13:07.219802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 21:13:07.372972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
model_path = "google/mt5-small"
model_path = '/kaggle/input/ivirit-mt5-result/mt5-small-finetuned_model'

In [5]:
# Load the pre-trained MT5 small model
model = MT5ForConditionalGeneration.from_pretrained(model_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
train_df = pd.read_csv('/kaggle/input/michael-ivrit-dataset/train_data.csv', index_col=None).dropna()
eval_df = pd.read_csv('/kaggle/input/michael-ivrit-dataset/train_data.csv', index_col=None).dropna()

In [22]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

# Load the pre-trained model and tokenizer
# model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
# tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

# Define the source and target languages
src_lang = "en"
tgt_lang = "fr"

# Example input text
sample = train_df[train_df['orig_text'] != train_df['text']].sample(1).to_dict()
ground_turth = sample['text']
text = sample['orig_text']

# Encode the input text and generate the target text
# input_ids = tokenizer.encode(f"{src_lang} </s> {text}", return_tensors="pt")
input_ids = tokenizer.encode(f"{text}", return_tensors="pt")
output_ids = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the output
print(f"Input: {text}")
print(f"Pred: {output_text}")
print(f"True: {ground_turth}")

Input: {33312: 'כפי שאמרנו, נפוליון ידע גם לשנות את תוכניותיו'}
Pred: {33312: 'כפי שאמרנו, נפוליון ידע גם לשנות את תוכניותיו'}
True: {33312: 'כפי שאמרנו, נפוליאון ידע גם לשנות את תוכניותיו'}


In [18]:
train_df[train_df['orig_text'] != train_df['text']].sample(1).to_dict()

{'Unnamed: 0': {30858: 30858},
 'uuid': {30858: 'NoTarbut/2023.05.20 פרק 121 – Fck Estimations/36'},
 'orig_text': {30858: 'היי כמה זמן כשכבר עזבתי והתחלנו להקים את תורי אז כסטארט-אפ'},
 'text': {30858: 'אחרי כמה זמן כשכבר עזבתי והתחלנו להקים את תורי אז כסטארט-אפ'}}

In [17]:
(train_df['orig_text'] != train_df['text']).mean()

0.5109533982691821